# Setting


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# kiwi 형태소 분석기
!pip install kiwipiepy

In [ ]:
# Sentence BERT
!pip install sentence_transformers

In [ ]:
import pandas as pd
import numpy as np
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from kiwipiepy import Kiwi
import warnings
from sklearn.cluster import KMeans

warnings.filterwarnings(action='ignore')

# Data Import

In [ ]:
# 기업 리뷰
review = pd.read_csv('/content/drive/MyDrive/TM_company_review.csv', encoding='euc-kr')

# 최종합격 후기
success = pd.read_csv('/content/drive/MyDrive/TM_company_success.csv', encoding='euc-kr')

# 기업 종합 리뷰
overall_review = pd.read_csv('/content/drive/MyDrive/TM_company_overall_review.csv', encoding='euc-kr')

# 직무별 합격자 스펙
success_spec = pd.read_csv('/content/drive/MyDrive/TM_success_spec.csv', encoding='euc-kr')
success_spec['task'] = success_spec['task'].apply(lambda x: x.strip())

In [ ]:
review

,company_name,good_review,bad_review,recommend
0,삼성전자,Good\n다니면 다닐 수록 대기업이라는 타이틀만 좋은 느낌이 드는 곳,Bad\n부바부 심하며 직원 복지가 조금씩 줄어 들고 있다는 느낌,추천
1,삼성전자,Good\n출퇴근 시간을 내 스케쥴 대로 할 수 있다. 눈치 안보고 휴가 쓸 수 있다.,Bad\n일이 너무 많다. 항상 일이 많다. 가끔은 일이 나를 절벽으로 몰아가고 있...,추천
2,삼성전자,Good\n직원들 체력 증진을 위해 헬스장이 잘 되어있으나 수영장은 대기가 필요하다,Bad\n경영진의 보여주기식 정치\n보여주기식 복지가 너무 많음.,추천
3,삼성전자,Good\n극강의 안정성이 최고입니다. HR 보장이 확고하여 휴가등이 보장됩니다,Bad\n사실 공무원 마인드가 있습니다. 망하지 않을 회사이기 때문입니다.,추천
4,삼성전자,Good\n눈치 없이 퇴근이 가능하나\n부서바이부서가 심하다. 월급 만족,Bad\n빡센 부서는 야근을 밥 먹듯이 하고 꼰대 문화가 있다..,추천
...,...,...,...,...
2222,SSG닷컴,Good\n스타벌스 할인 받을 수 있다는 점이 젤 좋은 것 같다,Bad\n구성원의 소중함을 모른다. 계속 퇴사자가 증가하는 중이다,비추
2223,SSG닷컴,Good\n임직원 사내 복지와 기업의 워라벨 문화는 좋은 편이에요,Bad\n신입사원 및 인턴 선발에 학력과 성적이 아주 중요합니다.,추천
2224,SSG닷컴,Good\n근무의 자유도가 높은편이고 대부분 정시퇴근 하는점. 휴일설정이 자유로운점,Bad\n승진하기가 힘든점 직원들 또는 고과주는사람 눈치봐야 하는점,추천
2225,SSG닷컴,"Good\n근무하면서 직원들끼리 부딪힐 일이 없음, 정시 칼퇴근 가능","Bad\n고과 잘받기가 매우 힘듬, 상사 눈치를 많이 봐야 됨",추천


In [ ]:
success

,company_name,tasks,tip,activity
0,금호타이어,영업/고객상담,"1. 대기업, 중견기업 인턴 1년 경력이 있습니다. 경력을 쌓으시며 얻은 경험, 지...",면접 스터디가 중요했습니다. 확실히 혼자 준비하면 게을러져 미루기 일쑤인데 팀을 짜...
1,현대로템,연구개발/설계,단순히 지원 직무에서 우대하는 관련 전공에 대한 지식 뿐만 아니라 여러 팀들과 협업...,"기업 홈페이지 및 유튜브, 현직자 리뷰 등을 통해 회사에 대한 이해도를 쌓았고 분기..."
2,포스코퓨처엠,생산/제조,"공고에서 프랑스어 가능자를 우대한다고 적혀있었기에, 예전에 따두었던 프랑스어 성적을...","저는 대기업 경영연구원에서 인턴을 하며, 매일매일 약 3시간씩 외신 뉴스 및 학회지..."
3,한림제약,연구개발/설계,자기소개서는 대부분 그렇듯 아주 특별한 경험을 갖고 있지는 않았다. 다만 내 경험을...,사실 특별하게 남들과 다른 경험을 갖고 있지는 않다. 오히려 남들보다 경험이 적을 ...
4,사조대림,경영/사무,"직무 관련 자격증을 다수 취득하였으며, 회사에 잘 적응해서 팀원들과 잘 어울릴 거 ...",학교 다닐 때 했던 동아리가 가장 도움이 된 거 같다. 동아리에서 운영진으로서 동아...
...,...,...,...,...
588,삼성바이오로직스,생산/제조,직무에 대한 분석과 이해가 필수적이라고 생각합니다. 저는 여러 route를 통해 직...,"취업스터디에 참여하게 되면, 동일한 시간에 혼자 공부하는 것의 몇 배의 효과를 얻게..."
589,메가스터디교육,경영/사무,취준을 하면서 무슨일을 하는지 잘 파악하지 못한채 무작정 총알을 채운다는 느낌으로 ...,"이진우의 손에 잡히는 경제 라디오를 매일 들었던게 가장 큰 도움이 된다, 취준은 꾸..."
590,엘엑스세미콘,연구개발/설계,"과는 정말 반도체 설계와 전혀 관련 없지만, 설계 직무에 대한 프로젝트, 인턴, 공...",취준 첫 입문당시에는 교내 취업 스터디의 도움을 받아 자소서 및 기업분석을 진행하였...
591,LG전자,경영/사무,"직무와 관련된 경험들을 많이 어필했고, 그러한 활동들과 자격증 취득을 통해 무엇을 ...",현직자를 만나는 것이 가장 도움이 되었음. 관련 부서가 있는지 알아보는 것과 회사 ...


In [ ]:
overall_review

,company_name,mood,salary,worktime,carrer,vision
0,삼성전자,84.1,87.7,85.3,85.8,83.6
1,현대자동차,82.9,85.0,85.4,83.1,81.5
2,GS칼텍스,81.8,83.9,82.0,83.3,82.6
3,SK에너지,85.0,86.6,84.1,85.6,83.8
4,기아,84.4,86.3,87.3,84.8,83.7
...,...,...,...,...,...,...
185,애경케미칼,86.4,83.5,83.5,82.4,79.6
186,DB하이텍,74.5,76.2,75.3,76.5,73.5
187,팜스코,90.0,80.0,82.5,80.0,75.0
188,현대백화점,79.2,79.4,79.4,77.3,77.6


In [ ]:
success_spec

,task,score,toeic,toeic_speaking,opic,certification,intern,activity,foreign,award
0,경영/사무,3.8,896,Lv.7,IH,1.4,0.6,0.4,0.1,0.3
1,영업/고객상담,3.7,919,Lv.7,IH,1.5,0.7,0.9,0.0,0.5
2,서비스,3.8,818,Lv.6,IH,1.6,0.3,0.2,0.0,0.1
3,무역/유통,3.8,930,Lv.7,IH,1.4,0.9,0.7,0.1,0.5
4,IT/인터넷,3.7,870,Lv.7,IM3,1.0,0.5,0.7,0.0,0.6
5,연구개발/설계,3.8,853,Lv.7,IM3,0.8,0.4,1.0,0.0,0.7
6,생산/제조,3.7,876,Lv.7,IH,1.4,0.3,0.7,0.0,0.5
7,건설,3.7,851,Lv.7,IH,1.5,0.5,0.6,0.0,0.2
8,마케팅/광고/홍보,3.9,922,Lv.8,IH,1.3,0.9,1.0,0.1,0.7


# EDA

In [ ]:
# 기업 리뷰가 존재하는 기업 수
len(np.unique(review.company_name))

167

In [ ]:
# 각 기업 당 리뷰 개수
review.company_name.value_counts().sort_values(ascending=False)

삼성전자         125
LG전자          70
LG디스플레이       62
현대자동차         61
삼성디스플레이       52
            ... 
장금상선           2
NH투자증권         1
태영건설           1
코오롱글로벌         1
HDC현대산업개발      1
Name: company_name, Length: 167, dtype: int64

In [ ]:
# 결측치 확인
review.isnull().sum()

company_name    0
good_review     0
bad_review      0
recommend       0
dtype: int64

In [ ]:
# 합격후기가 존재하는 기업 수
len(np.unique(success.company_name))

344

In [ ]:
# 합격후기 내 직무 종류
success.tasks.value_counts().sort_values(ascending=False)

경영/사무        113
생산/제조        112
연구개발/설계       91
영업/고객상담       79
IT/인터넷        73
마케팅/광고/홍보     43
무역/유통         36
건설            19
서비스           10
전문/특수직         8
미디어            5
디자인            4
Name: tasks, dtype: int64

In [ ]:
# 각 기업 당 취업후기 개수 (상위 20개)
success.company_name.value_counts().sort_values(ascending=False).head(20)

삼성전자        37
엘지에너지솔루션    15
LG디스플레이     12
SK하이닉스      10
현대모비스        9
현대자동차        9
포스코홀딩스       8
한국철도공사       7
엘아이지넥스원      7
현대건설         6
LG화학         6
교보생명보험       5
삼성바이오로직스     5
롯데케미칼        5
현대오토에버       4
GS리테일        4
LG전자         4
KT           4
KT DS        4
코오롱인더스트리     4
Name: company_name, dtype: int64

In [ ]:
# 결측치 확인
success.isnull().sum()

company_name    0
tasks           0
tip             0
activity        0
dtype: int64

In [ ]:
# 종합 리뷰가 존재하는 기업 수
len(np.unique(overall_review.company_name))

190

In [ ]:
# 결측치 확인
overall_review.isnull().sum()

company_name    0
mood            0
salary          0
worktime        0
carrer          0
vision          0
dtype: int64

# Preprocessing

## Review data

In [ ]:
# 아래 리뷰 속 Good(Bad)\n 제거 및 개행문자와 '.'을 기준으로 문장 분리
review.loc[4, ['good_review', 'bad_review']]

good_review    Good\n눈치 없이 퇴근이 가능하나\n부서바이부서가 심하다. 월급 만족
bad_review         Bad\n빡센 부서는 야근을 밥 먹듯이 하고 꼰대 문화가 있다..
Name: 4, dtype: object

In [ ]:
review['good_review'][4]

'Good\n눈치 없이 퇴근이 가능하나\n부서바이부서가 심하다. 월급 만족'

In [ ]:
def split_text(text):
  words = re.split(r'[.\n]', text)[1:]
  words = [word.strip() for word in words if word.strip()]
  words = [word for word in words if len(word) >= 2]
  return words

In [ ]:
review['good_review'] = review['good_review'].apply(split_text)
review['bad_review'] = review['bad_review'].apply(split_text)

In [ ]:
review

,company_name,good_review,bad_review,recommend
0,삼성전자,[다니면 다닐 수록 대기업이라는 타이틀만 좋은 느낌이 드는 곳],[부바부 심하며 직원 복지가 조금씩 줄어 들고 있다는 느낌],추천
1,삼성전자,"[출퇴근 시간을 내 스케쥴 대로 할 수 있다, 눈치 안보고 휴가 쓸 수 있다]","[일이 너무 많다, 항상 일이 많다, 가끔은 일이 나를 절벽으로 몰아가고 있다는 생...",추천
2,삼성전자,[직원들 체력 증진을 위해 헬스장이 잘 되어있으나 수영장은 대기가 필요하다],"[경영진의 보여주기식 정치, 보여주기식 복지가 너무 많음]",추천
3,삼성전자,"[극강의 안정성이 최고입니다, HR 보장이 확고하여 휴가등이 보장됩니다]","[사실 공무원 마인드가 있습니다, 망하지 않을 회사이기 때문입니다]",추천
4,삼성전자,"[눈치 없이 퇴근이 가능하나, 부서바이부서가 심하다, 월급 만족]",[빡센 부서는 야근을 밥 먹듯이 하고 꼰대 문화가 있다],추천
...,...,...,...,...
2222,SSG닷컴,[스타벌스 할인 받을 수 있다는 점이 젤 좋은 것 같다],"[구성원의 소중함을 모른다, 계속 퇴사자가 증가하는 중이다]",비추
2223,SSG닷컴,[임직원 사내 복지와 기업의 워라벨 문화는 좋은 편이에요],[신입사원 및 인턴 선발에 학력과 성적이 아주 중요합니다],추천
2224,SSG닷컴,"[근무의 자유도가 높은편이고 대부분 정시퇴근 하는점, 휴일설정이 자유로운점]",[승진하기가 힘든점 직원들 또는 고과주는사람 눈치봐야 하는점],추천
2225,SSG닷컴,"[근무하면서 직원들끼리 부딪힐 일이 없음, 정시 칼퇴근 가능]","[고과 잘받기가 매우 힘듬, 상사 눈치를 많이 봐야 됨]",추천


In [ ]:
review.good_review[4]

['눈치 없이 퇴근이 가능하나', '부서바이부서가 심하다', '월급 만족']

In [ ]:
# explode 메서드를 통해 리스트 내 한 문장을 하나의 row로 표현
good_review = review[['company_name', 'good_review']]
good_review = good_review.explode('good_review')
good_review.reset_index(inplace=True, drop=True)
good_review

,company_name,good_review
0,삼성전자,다니면 다닐 수록 대기업이라는 타이틀만 좋은 느낌이 드는 곳
1,삼성전자,출퇴근 시간을 내 스케쥴 대로 할 수 있다
2,삼성전자,눈치 안보고 휴가 쓸 수 있다
3,삼성전자,직원들 체력 증진을 위해 헬스장이 잘 되어있으나 수영장은 대기가 필요하다
4,삼성전자,극강의 안정성이 최고입니다
...,...,...
4252,SSG닷컴,임직원 사내 복지와 기업의 워라벨 문화는 좋은 편이에요
4253,SSG닷컴,근무의 자유도가 높은편이고 대부분 정시퇴근 하는점
4254,SSG닷컴,휴일설정이 자유로운점
4255,SSG닷컴,"근무하면서 직원들끼리 부딪힐 일이 없음, 정시 칼퇴근 가능"


In [ ]:
bad_review = review[['company_name', 'bad_review']]
bad_review = bad_review.explode('bad_review')
bad_review.reset_index(inplace=True, drop=True)
bad_review

,company_name,bad_review
0,삼성전자,부바부 심하며 직원 복지가 조금씩 줄어 들고 있다는 느낌
1,삼성전자,일이 너무 많다
2,삼성전자,항상 일이 많다
3,삼성전자,가끔은 일이 나를 절벽으로 몰아가고 있다는 생각이 들 때가 많다
4,삼성전자,경영진의 보여주기식 정치
...,...,...
4242,SSG닷컴,계속 퇴사자가 증가하는 중이다
4243,SSG닷컴,신입사원 및 인턴 선발에 학력과 성적이 아주 중요합니다
4244,SSG닷컴,승진하기가 힘든점 직원들 또는 고과주는사람 눈치봐야 하는점
4245,SSG닷컴,"고과 잘받기가 매우 힘듬, 상사 눈치를 많이 봐야 됨"


In [ ]:
# 각 회사 별 긍부정 리뷰의 수가 다름
# 데이터프레임을 따로 나누어 분석 진행
good_review[good_review.company_name=='삼성전자'].shape, bad_review[bad_review.company_name=='삼성전자'].shape

((253, 2), (248, 2))

In [ ]:
# 삼성전자에 대한 긍정 리뷰 문장 샘플 10개
good_review[good_review.company_name=='삼성전자'].sample(10, random_state=21)

,company_name,good_review
60,삼성전자,무엇보다 좋은 것은 3끼 무료인 밥이겠지요
141,삼성전자,휴가 쓰기 편하고 솔직히 제일 좋은건 어디가서 직업 말할때 그나마 평타는 친다고 생각함
249,삼성전자,안정적인 대기업 야근수당 보장 및 식대 및 복지는 매우 좋다
130,삼성전자,"쾌적한 근무 환경과 사내 분위기, 미래 성장 가능성, 경력 개발"
20,삼성전자,완벽한 근무환경 통합버스 운영으로 시간절약 높은 만족도로 일하고 있으며 자기개발 환...
158,삼성전자,회사와 함께 성장이 가능하다
108,삼성전자,지하철역이 근처에 있어서 교통이 편리하다
140,삼성전자,초봉이 높고 어디가서 회사로 꿀릴 일이 적다
114,삼성전자,밥은 회사중에 최고인 것같아
123,삼성전자,복지 및 인센은 최고인거 같습니다


In [ ]:
# 이후 클러스터링을 위해, good_review, bad_review의 row 개수가 5개 미만인 기업은 제거
for c in review.company_name.unique():
  if good_review[good_review.company_name==c].shape[0] < 5 or bad_review[bad_review.company_name==c].shape[0] < 5:
    good_review = good_review[good_review.company_name!=c]
    bad_review = bad_review[bad_review.company_name!=c]

good_review.shape, bad_review.shape

((4190, 2), (4185, 2))

## Success

In [ ]:
success

,company_name,tasks,tip,activity
0,금호타이어,영업/고객상담,"1. 대기업, 중견기업 인턴 1년 경력이 있습니다. 경력을 쌓으시며 얻은 경험, 지...",면접 스터디가 중요했습니다. 확실히 혼자 준비하면 게을러져 미루기 일쑤인데 팀을 짜...
1,현대로템,연구개발/설계,단순히 지원 직무에서 우대하는 관련 전공에 대한 지식 뿐만 아니라 여러 팀들과 협업...,"기업 홈페이지 및 유튜브, 현직자 리뷰 등을 통해 회사에 대한 이해도를 쌓았고 분기..."
2,포스코퓨처엠,생산/제조,"공고에서 프랑스어 가능자를 우대한다고 적혀있었기에, 예전에 따두었던 프랑스어 성적을...","저는 대기업 경영연구원에서 인턴을 하며, 매일매일 약 3시간씩 외신 뉴스 및 학회지..."
3,한림제약,연구개발/설계,자기소개서는 대부분 그렇듯 아주 특별한 경험을 갖고 있지는 않았다. 다만 내 경험을...,사실 특별하게 남들과 다른 경험을 갖고 있지는 않다. 오히려 남들보다 경험이 적을 ...
4,사조대림,경영/사무,"직무 관련 자격증을 다수 취득하였으며, 회사에 잘 적응해서 팀원들과 잘 어울릴 거 ...",학교 다닐 때 했던 동아리가 가장 도움이 된 거 같다. 동아리에서 운영진으로서 동아...
...,...,...,...,...
588,삼성바이오로직스,생산/제조,직무에 대한 분석과 이해가 필수적이라고 생각합니다. 저는 여러 route를 통해 직...,"취업스터디에 참여하게 되면, 동일한 시간에 혼자 공부하는 것의 몇 배의 효과를 얻게..."
589,메가스터디교육,경영/사무,취준을 하면서 무슨일을 하는지 잘 파악하지 못한채 무작정 총알을 채운다는 느낌으로 ...,"이진우의 손에 잡히는 경제 라디오를 매일 들었던게 가장 큰 도움이 된다, 취준은 꾸..."
590,엘엑스세미콘,연구개발/설계,"과는 정말 반도체 설계와 전혀 관련 없지만, 설계 직무에 대한 프로젝트, 인턴, 공...",취준 첫 입문당시에는 교내 취업 스터디의 도움을 받아 자소서 및 기업분석을 진행하였...
591,LG전자,경영/사무,"직무와 관련된 경험들을 많이 어필했고, 그러한 활동들과 자격증 취득을 통해 무엇을 ...",현직자를 만나는 것이 가장 도움이 되었음. 관련 부서가 있는지 알아보는 것과 회사 ...


In [ ]:
# Kkma 성능이 좋지 않아 아래 함수 사용
def split_text(text):
  words = re.split(r'[.\n]', text)
  words = [word.strip() for word in words if word.strip()]
  words = [word for word in words if len(word) >= 2]
  return words

success['tip'] = success['tip'].apply(split_text)
success['activity'] = success['activity'].apply(split_text)

In [ ]:
# 같은 방법으로 explode
tip = success[['tasks', 'tip']]
activity = success[['tasks', 'activity']]

tip = tip.explode('tip')
tip.reset_index(inplace=True, drop=True)
activity = activity.explode('activity')
activity.reset_index(inplace=True, drop=True)

In [ ]:
tip

,tasks,tip
0,영업/고객상담,"대기업, 중견기업 인턴 1년 경력이 있습니다"
1,영업/고객상담,"경력을 쌓으시며 얻은 경험, 지식 등을 잘 정리하시고 필살기 꼭 쓰시길 바랍니다"
2,영업/고객상담,"솔직하게 제 생각을 얘기한 점, 현직자 인터뷰하며 직무에 대한 사전 준비가 합격으로..."
3,영업/고객상담,저는 이전 취준 때도 최종 면접까지 많이 갔습니다
4,영업/고객상담,"이런 경험이 있다 보니 막상 면접 때, 전혀 떨리지도 않았습니다"
...,...,...
2082,경영/사무,요즘 시국은 진짜 운이 더 많이 작용한다고 생각합니다 ㅠ
2083,경영/사무,"ㅠ 이번에 합격한 곳도 전혀 생각지 않았던 업종과 직종이었는데, 서류 공고를 발견하..."
2084,경영/사무,면접도 그 경험이 이 기업에서 실제 하는 업무라 일치해 좋게 봐주셔서 이것만 '기5...
2085,경영/사무,"하지만 '기'로 '운'을 만들어 낼 수 있고, '운'으로 '기'를 만들어 낼 수 있..."


In [ ]:
activity

,tasks,activity
0,영업/고객상담,면접 스터디가 중요했습니다
1,영업/고객상담,확실히 혼자 준비하면 게을러져 미루기 일쑤인데 팀을 짜니까 책임감이 생겨 준비를 열...
2,영업/고객상담,본인의 의지가 약하다면 그런 활동의 힘을 빌리는 게 중요할 거라 생각합니다
3,영업/고객상담,또한 공고 뜨면 당연히 기업 분석을 하지만 그것과 '나'를 연결하는 과정이 중요한 ...
4,연구개발/설계,"기업 홈페이지 및 유튜브, 현직자 리뷰 등을 통해 회사에 대한 이해도를 쌓았고 분기..."
...,...,...
2015,경영/사무,"관련 부서가 있는지 알아보는 것과 회사 분위기, 어떤 신입을 좋아하는지, 기타 연봉..."
2016,경영/사무,또한 자기소개서 등도 보여드리면서 필요한 점들을 보완하는 계기가 되었음
2017,경영/사무,"현직자 만나서 인터뷰 하고, 면접스터디 꾸준히 하면 도움되겠지만 코로나 등 제약 때..."
2018,경영/사무,"(솔직히 현직자도 자기 회사를 100% 아는게 아니고, 시간도 많이 뺏기는 느낌이라..."


## Sentence Embedding with SBERT

In [ ]:
# 100가지 언어를 지원(한국어 포함)하는 다국어 BERT BASE 모델
# model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

# 서울대학교의 KR-SBERT 모델 사용(평가 결과 기반)
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [ ]:
# SBERT를 활용한 문장 임베딩
good_review['embedding'] = good_review.apply(lambda x: model.encode(x.good_review), axis = 1)

In [ ]:
# 하나의 문장이 768 차원의 벡터로 임베딩
len(good_review.embedding[0])

768

In [ ]:
# 나머지 텍스트도 임베딩 진행
bad_review['embedding'] = bad_review.apply(lambda x: model.encode(x.bad_review), axis = 1)
tip['embedding'] = tip.apply(lambda x: model.encode(x.tip), axis = 1)
activity['embedding'] = activity.apply(lambda x: model.encode(x.activity), axis = 1)

# TextRank with Sentence Embedding

In [ ]:
embedding_dim = 768

# 문장간 유사도 행렬 계산
def similarity_matrix(sentence_embedding):
  sim_mat = np.zeros([len(sentence_embedding), len(sentence_embedding)])
  for i in range(len(sentence_embedding)):
      for j in range(len(sentence_embedding)):
        sim_mat[i][j] = cosine_similarity(sentence_embedding[i].reshape(1, embedding_dim),
                                          sentence_embedding[j].reshape(1, embedding_dim))[0,0]
  return sim_mat

# 유사도 행렬 기반 문장 별 점수 계산
def calculate_score(sim_matrix):
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)
    return scores

# 점수가 가장 높은 문장 1개의 인덱스 계산
# 해당하는 인덱스는 전체 텍스트를 요약하는 문장의 인덱스
def calc_top_index(score):
  top_index = []
  temp_score = sorted(score.items(), key=lambda x: x[1], reverse=True)[:]
  top_index.append(temp_score[0][0])
  return top_index

# 전체 텍스트를 5개의 클러스터로 분할
# 이후 calc_top_index로 각 클러스터 당 대표하는 1개 인덱스 계산
def text_clustering(data):
  vector_data = data['embedding'].tolist()
  X = [x for x in vector_data]
  X = np.array(X)

  kmeans = KMeans(n_clusters=5, random_state=21)
  kmeans.fit(X)
  labels = kmeans.labels_

  data['cluster'] = labels
  return data

In [ ]:
def review_summarization(data, review_type):
  company_review = pd.DataFrame(columns=['company_name', 'embeddings'])

  embedding_list = []
  temp_review = data
  for embedding in temp_review['embedding']:
    embedding_list.append(embedding)

  company_review['company_name'] = [temp_review.company_name]
  company_review['embeddings'] = [embedding_list]

  company_review.reset_index(inplace=True, drop=True)
  company_review['SimMatrix']= company_review['embeddings'].apply(similarity_matrix)
  company_review['score'] = company_review['SimMatrix'].apply(calculate_score)
  company_review['top_index'] = company_review['score'].apply(calc_top_index)

  temp_review = data.reset_index(drop=True)
  index = company_review['top_index']
  review_list = []
  if review_type.lower() == 'good':
    for i in index[0]:
      review_list.append(temp_review.loc[i, 'good_review'])
  elif review_type.lower() == 'bad':
    for i in index[0]:
      review_list.append(temp_review.loc[i, 'bad_review'])
  return review_list

In [ ]:
def success_summarization(tip, activity, task_name):
  task_tip = pd.DataFrame(columns=['task', 'embeddings'])
  task_activity = pd.DataFrame(columns=['task', 'embeddings'])

  tip_embedding_list = []
  act_embedding_list = []

  temp_tip, temp_act = tip, activity
  for embedding in temp_tip['embedding']:
    tip_embedding_list.append(embedding)
  for embedding in temp_act['embedding']:
    act_embedding_list.append(embedding)

  temp_tip = pd.DataFrame(columns=['task', 'embeddings'])
  temp_tip['task'] = [task_name]
  temp_tip['embeddings'] = [tip_embedding_list]
  task_tip = pd.concat([task_tip, temp_tip])

  temp_activity = pd.DataFrame(columns=['task', 'embeddings'])
  temp_activity['task'] = [task_name]
  temp_activity['embeddings'] = [act_embedding_list]
  task_activity = pd.concat([task_activity, temp_activity])

  task_tip.reset_index(inplace=True, drop=True)
  task_activity.reset_index(inplace=True, drop=True)

  task_tip['SimMatrix']= task_tip['embeddings'].apply(similarity_matrix)
  task_tip['score'] = task_tip['SimMatrix'].apply(calculate_score)
  task_tip['top_index'] = task_tip['score'].apply(calc_top_index)

  task_activity['SimMatrix']= task_activity['embeddings'].apply(similarity_matrix)
  task_activity['score'] = task_activity['SimMatrix'].apply(calculate_score)
  task_activity['top_index'] = task_activity['score'].apply(calc_top_index)

  temp_tip, temp_act = tip[tip.tasks==task_name].reset_index(drop=True), activity[activity.tasks==task_name].reset_index(drop=True)
  tip_index, act_index = task_tip[task_tip.task==task_name]['top_index'], task_activity[task_activity.task==task_name]['top_index']

  tip_list = []
  act_list = []

  for i in tip_index[0]:
    tip_list.append(temp_tip.loc[i, 'tip'])

  for i in act_index[0]:
    act_list.append(temp_act.loc[i, 'activity'])
  return tip_list, act_list

# Keyphrase Extraction with Sentence Embedding, MMR

$\text{MMR} := {\text{argmax} \atop D_i \in R}[\lambda \cdot \text{Sim}_1(D_i, Q) - (1-\lambda)\; {\text{max} \atop D_j \in S} \; \text{Sim}_2 (D_i, D_j) ]$

$Q$는 전체 문서, $D_i, D_j$는 각 토큰, $\lambda$는 0에서 1사이 값을 가지며, 유사도 $\text{Sim}_i$는 코사인 유사도를 사용한다.

오른쪽 식이 최대가 되기 위해서는 어떤 토큰과 전체 문서와의 유사도가 높아야 하고, 또한 그 토큰과 다른 토큰과의 유사도는 낮아야 한다.

두번째 항을 추가함으로, 출력되는 토큰(여기서는 Keyphrase)간의 유사도가 낮아져 다양성을 가진 키워드를 얻을 수 있다.



In [ ]:
# Kiwi 형태소 분석기를 사용한 품사 태깅
# 명사만 추출
def noun_extractor(text):
  kiwi = Kiwi()
  results = []
  result = kiwi.analyze(text)
  for token, pos, _, _ in result[0][0]:
    # 명사 또는 외국어(SL) 관련 토큰만 추출
    if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
      results.append(token)
  return results

# 키워드의 다양성을 위한 mmr 알고리즘
# diversity = 1 - lambda로 생각할 수 있음
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        try:
          candidate_similarities = word_doc_similarity[candidates_idx, :]
          target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

          # MMR을 계산
          mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
          mmr_idx = candidates_idx[np.argmax(mmr)]

          # keywords & candidates를 업데이트
          keywords_idx.append(mmr_idx)
          candidates_idx.remove(mmr_idx)
        except:
          break

    return [words[idx] for idx in keywords_idx]

# 입력된 텍스트에 대해 10개의 키워드를 출력하는 함수
def extract_keywords(text):
  # 명사 추출 후 명사 토큰만 사용
	n_gram_range = (1, 1)
	nouns = noun_extractor(text)
	text = ' '.join(nouns)

  # n-gram을 고려한 CountVectorizer
	count = CountVectorizer(ngram_range=n_gram_range).fit([text])
	candidates = count.get_feature_names_out()

  # SentenceBERT를 사용한 인코딩
	doc_embedding = model.encode([text])
	candidate_embeddings = model.encode(candidates)

  # mmr 알고리즘을 적용하여 다양성을 가진 10개의 키워드 추출
	return mmr(doc_embedding, candidate_embeddings, candidates, top_n=10, diversity=0.4)

In [ ]:
def extract_keywords_from_review(data, review_type):
  if review_type == 'good':
    review_text = '. '.join(data.good_review)
    return extract_keywords(review_text)

  elif review_type == 'bad':
    review_text = '. '.join(data.bad_review)
    return extract_keywords(review_text)

def extract_keywords_from_success(data, success_type):
  if success_type == 'tip':
    success_text = '. '.join(data.tip)
    return extract_keywords(success_text)

  elif success_type == 'activity':
    success_text = '. '.join(data.activity)
    return extract_keywords(success_text)

# Providing Employment Information

In [ ]:
def print_company_list():
  print('검색 가능한 기업 종류는 다음과 같습니다', end='\n\n')
  n_iter = len(good_review.company_name.unique()) // 7 + 1
  for i in range(n_iter):
    if i != n_iter:
      print(good_review.company_name.unique()[7*i:7*(i+1)])
    elif i == n_iter:
      print(good_review.company_name.unique()[7*i:])

def print_task_list():
  print('검색 가능한 직무 종류는 다음과 같습니다', end='\n\n')
  print(success.tasks.unique()[:6])
  print(success.tasks.unique()[6:])

In [ ]:
def print_company_review(company_name):
  '''
  Parameter
  ------------
  company_name: str, 회사 이름

  Output
  ------------
  company information: 회사의 종합적 리뷰 및 good 리뷰, bad 리뷰에 대한 요약 제시
  '''
  print('-'*200, end='\n\n')
  print(company_name, end='\n\n')

  company_overall_review = overall_review[overall_review.company_name==company_name]
  company_review = review[review.company_name==company_name]
  company_good_review = good_review[good_review.company_name==company_name]
  company_bad_review = bad_review[bad_review.company_name==company_name]
  company_success = success[success.company_name==company_name]

  # 회사의 종합적 리뷰 출력
  # shape[0]이 0이라면 해당 기업에 대한 종합 리뷰가 존재하지 않음
  if company_overall_review.shape[0] != 0:
    print('해당 기업의 종합적인 리뷰 점수는 다음과 같습니다.')
    print('   1) 조직문화/분위기:  %.1f' %company_overall_review.mood.iloc[0])
    print('   2) 연봉/복지:  %.1f' %company_overall_review.salary.iloc[0])
    print('   3) 근무시간/휴가:  %.1f' %company_overall_review.worktime.iloc[0])
    print('   4) 커리어/성장:  %.1f' %company_overall_review.carrer.iloc[0])
    print('   5) 경영진/비전:  %.1f' %company_overall_review.vision.iloc[0])
    print(end='\n\n')

  # 회사의 리뷰 중 추천 비율 출력
  total_review_cnts = company_review.shape[0]
  recommend_cnts = company_review.recommend.str.contains('추천').sum()
  print('전체 리뷰 %d개의 리뷰 중 %d개의 추천 리뷰가, %d개의 비추천 리뷰가 존재합니다.' %(total_review_cnts, recommend_cnts, total_review_cnts - recommend_cnts), end='\n')
  print('전체 리뷰 중 추천 리뷰의 비율은 %.2f%%입니다.' %(recommend_cnts/total_review_cnts*100), end='\n\n\n')

  # 클러스터링
  company_good_review = text_clustering(company_good_review)
  company_bad_review = text_clustering(company_bad_review)

  # Good 리뷰 요약
  print('해당 기업의 Good review 요약')
  for c in range(5):
    good_review_list = review_summarization(company_good_review[company_good_review.cluster==c], 'good')
    for i in range(1):
      try:
        print('- ', good_review_list[i])
      except:
        break

  print()
  print('해당 기업의 Good review에 대한 키워드')
  good_keyword_list = extract_keywords_from_review(company_good_review, 'good')
  for w in good_keyword_list:
    print(w, end='    ')
  print(end='\n\n')

  # Bad 리뷰 요약
  print('해당 기업의 Bad review 요약')
  for c in range(5):
    bad_review_list = review_summarization(company_bad_review[company_bad_review.cluster==c], 'bad')
    for i in range(1):
      try:
        print('- ', bad_review_list[i])
      except:
        break
  print()

  print('해당 기업의 Bad review에 대한 키워드')
  bad_keyword_list = extract_keywords_from_review(company_bad_review, 'bad')
  for w in bad_keyword_list:
    print(w, end='    ')

  print(end='\n\n\n')
  n_company_success = len(company_success)
  print('해당 기업의 최종합격 후기가 %d개 존재합니다.' %(len(company_success)))

  if (n_company_success > 0):
    print('다음은 해당 기업의 최종합격 후기 예시입니다.', end='\n\n')

    print('직무: %s' %(company_success.tasks.iloc[0]), end='\n\n')
    print('1. 본인이 생각한 합격 이유와 합격 Tip')
    for i in range(len(company_success.tip.iloc[0])):
      print(company_success.tip.iloc[0][i], end='\n')
    print()
    print('2.취업준비에 가장 도움이 되었던 활동')
    for i in range(len(company_success.activity.iloc[0])):
      print(company_success.activity.iloc[0][i], end='\n')

  # End
  print()
  print('-'*200)

In [ ]:
def print_task_review(task_name):
  '''
  Parameter
  ------------
  task_name: str, 직무 이름

  Output
  ------------
  task information: 해당 직무에 대한 정보, 합격자들의 스펙 및 합격 Tip과 도움이 되었던 활동 요약 제시
  '''
  print('-'*200, end='\n\n')

  print('직무: ', task_name, end='\n\n')

  task_success = success[success.tasks==task_name]
  task_tip = tip[tip.tasks==task_name]
  task_activity = activity[activity.tasks==task_name]
  task_success_spec = success_spec[success_spec.task==task_name]

  spec_list = ['경영/사무', '영업/고객상담', '서비스', '무역/유통', 'IT/인터넷', '연구개발/설계', '생산/제조', '건설', '마케팅/광고/홍보']

  print('해당 직무에 대한 합격 후기가 %d개 존재합니다' %(task_success.shape[0]), end='\n\n')

  if task_name in spec_list:
    print('해당 직무 합격자의 평균 스펙은 다음과 같습니다.')
    print('   1) 평균 학점은 %.1f점 입니다.' %task_success_spec.score.iloc[0])
    print('   2) 평균 토익 점수는 %d점 입니다.' %task_success_spec.toeic.iloc[0])
    print('   3) 평균 토익 스피킹 레벨은 %s 입니다.' %task_success_spec.toeic_speaking.iloc[0])
    print('   4) 평균 오픽 레벨은 %s 입니다.' %task_success_spec.opic.iloc[0])
    print('   5) 평균 자격증 개수는 %.1f개 입니다.' %task_success_spec.certification.iloc[0])
    print('   6) 평균 인턴 경험은 %.1f회 입니다.' %task_success_spec.intern.iloc[0])
    print('   7) 평균 대외 활동 경험은 %.1f회 입니다.' %task_success_spec.activity.iloc[0])
    print('   8) 평균 해외 활동 경험은 %.1f회 입니다.' %task_success_spec.foreign.iloc[0])
    print('   9) 평균 수상 내역은 %.1f회 입니다.' %task_success_spec.award.iloc[0])
  else:
    print('해당 직무에 대한 합격자 분석이 존재하지 않습니다.')
  print(end='\n\n')

  # 클러스터링
  task_tip = text_clustering(task_tip)
  task_activity = text_clustering(task_activity)

  tip_list_1, activity_list_1 = success_summarization(task_tip[task_tip.cluster==0], task_activity[task_activity.cluster==0], task_name)
  tip_list_2, activity_list_2 = success_summarization(task_tip[task_tip.cluster==1], task_activity[task_activity.cluster==1], task_name)
  tip_list_3, activity_list_3 = success_summarization(task_tip[task_tip.cluster==2], task_activity[task_activity.cluster==2], task_name)
  tip_list_4, activity_list_4 = success_summarization(task_tip[task_tip.cluster==3], task_activity[task_activity.cluster==3], task_name)
  tip_list_5, activity_list_5 = success_summarization(task_tip[task_tip.cluster==4], task_activity[task_activity.cluster==4], task_name)

  print('본인이 생각한 최종합격 이유와 합격 Tip 요약')
  print('- ', tip_list_1[0])
  print('- ', tip_list_2[0])
  print('- ', tip_list_3[0])
  print('- ', tip_list_4[0])
  print('- ', tip_list_5[0])
  print()
  print('해당 직무의 합격 이유와 합격 Tip에 대한 키워드')
  tip_keyword_list = extract_keywords_from_success(task_tip, 'tip')
  for w in tip_keyword_list:
    print(w, end='    ')
  print(end='\n\n')

  print('취업 준비시 가장 도움이 된 활동 요약')
  print('- ', activity_list_1[0])
  print('- ', activity_list_2[0])
  print('- ', activity_list_3[0])
  print('- ', activity_list_4[0])
  print('- ', activity_list_5[0])
  print(end='\n\n')

  print('해당 직무의 가장 도움이 된 활동에 대한 키워드')
  act_keyword_list = extract_keywords_from_success(task_activity, 'activity')
  for w in act_keyword_list:
    print(w, end='    ')
  print()

  # End
  print()
  print('-'*200)

## Application

In [ ]:
print_company_list()

검색 가능한 기업 종류는 다음과 같습니다

['삼성전자' '현대자동차' 'GS칼텍스' '기아' 'S-OIL' 'SK하이닉스' '삼성생명보험']
['현대모비스' '에이치디현대오일뱅크' '포스코인터내셔널' '삼성디스플레이' 'LG전자' '삼성물산' '쿠팡']
['삼성화재해상보험' 'LG디스플레이' '현대제철' 'LG화학' '한국투자증권' '현대글로비스' '한화생명보험']
['교보생명보험' '현대해상화재보험' 'DB손해보험' 'LG이노텍' 'HMM' 'KT' '농협은행']
['삼성SDI' '미래에셋증권' '롯데케미칼' '이마트' '대한항공' 'LG유플러스' 'SK텔레콤']
['현대건설' '농협경제지주' '엘지에너지솔루션' 'GS리테일' '한화솔루션' '대우건설' '에이치디현대중공업']
['한국GM' '롯데쇼핑' 'GS건설' '포스코홀딩스' '한화손해보험' 'CJ대한통운' '포스코이앤씨']
['고려아연' '키움증권' 'CJ제일제당' '동국제강' '현대트랜시스' '현대엔지니어링' '삼성전기']
['현대위아' '삼성엔지니어링' 'SK네트웍스' 'SK에코플랜트' '롯데건설' '삼성중공업' '팬오션']
['아시아나항공' '호텔롯데' '네이버' '효성TNC' '두산에너빌리티' '삼성SDS' '금호석유화학']
['한화오션' '코리아세븐' '현대삼호중공업' '흥국생명보험' 'LG CNS' 'GS글로벌' '현대캐피탈']
['호텔신라' 'LS전선' 'SK브로드밴드' '한화' '코오롱인더스트리' 'LG생활건강' '현대미포조선']
['KT&G' '삼성카드' '비씨카드' '한국타이어앤테크놀로지' '삼성전자판매' '케이지모빌리티' 'SK C&C']
['롯데하이마트' '롯데글로벌로지스' '미래에셋생명보험' '현대로템' '포스코퓨처엠' 'CJ ENM' '세메스']
['아모레퍼시픽' '코웨이' 'CJ올리브영' '현대카드' '한국항공우주산업' 'KCC' '하']
['에이치엘만도' '롯데칠성음료' '에스씨케이컴퍼니' '효성중공업' 'HD현대건설기계' '롯데렌탈' '카카오']
['롯데웰푸드' '삼성바

In [ ]:
print_company_review('삼성전자')

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

삼성전자

해당 기업의 종합적인 리뷰 점수는 다음과 같습니다.
   1) 조직문화/분위기:  84.1
   2) 연봉/복지:  87.7
   3) 근무시간/휴가:  85.3
   4) 커리어/성장:  85.8
   5) 경영진/비전:  83.6


전체 리뷰 125개의 리뷰 중 108개의 추천 리뷰가, 17개의 비추천 리뷰가 존재합니다.
전체 리뷰 중 추천 리뷰의 비율은 86.40%입니다.


해당 기업의 Good review 요약
-  근무환경과 복지가 좋은 편이다
-  연봉이 높고 복지도 나쁘지 않으며 자율출퇴근제 및 연차사용 자유로움 굿 사람들 좋음
-  통근버스있어서 좋다
-  무엇보다 좋은 것은 3끼 무료인 밥이겠지요
-  초봉이 높고 어디가서 회사로 꿀릴 일이 적다

해당 기업의 Good review에 대한 키워드
유연근무제    안전    동료애    육휴    진급    여름휴가    성과급    재택    헬스장    퇴근    

해당 기업의 Bad review 요약
-  연봉이 성과급에 의존하는게 크다
-  가끔 업무강도가 너무 높고 야근이 갑자기 생겨서 불편하다
-  경직되고 비효율적인 프로세스가 많으며 지시하는 조직이 너무 많다
-  나쁜 점이라기보다는 회사가 커서 어쩔 수 없는 부분이라 생각합니다
-  교대인력이 부족해 한명이 연차쓰면 앞뒤로 커버함

해당 기업의 Bad review에 대한 키워드
직원    힘듦    관료주의    워라밸    디버깅    상명하복    휴가    과중    야근    smd    


해당 기업의 최종합격 후기가 37개 존재합니다.
다음은 해당 

In [ ]:
print_task_list()

검색 가능한 직무 종류는 다음과 같습니다

['영업/고객상담' '연구개발/설계' '생산/제조' '경영/사무' '전문/특수직' '디자인']
['IT/인터넷' '마케팅/광고/홍보' '무역/유통' '건설' '서비스' '미디어']


In [ ]:
print_task_review('IT/인터넷')

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

직무:  IT/인터넷

해당 직무에 대한 합격 후기가 73개 존재합니다

해당 직무 합격자의 평균 스펙은 다음과 같습니다.
   1) 평균 학점은 3.7점 입니다.
   2) 평균 토익 점수는 870점 입니다.
   3) 평균 토익 스피킹 레벨은 Lv.7 입니다.
   4) 평균 오픽 레벨은 IM3 입니다.
   5) 평균 자격증 개수는 1.0개 입니다.
   6) 평균 인턴 경험은 0.5회 입니다.
   7) 평균 대외 활동 경험은 0.7회 입니다.
   8) 평균 해외 활동 경험은 0.0회 입니다.
   9) 평균 수상 내역은 0.6회 입니다.


본인이 생각한 최종합격 이유와 합격 Tip 요약
-  이처럼 경험을 직무 중심으로 어떻게 풀어낼지에 대한 심층적인 고민이 필요하고, 또한 그 과정에 무엇을 배웠는지 배운 역량을 어떻게 앞으로 기업 입장에서 도움을 줄 수 있는 지를 어필해서 좋은 칭찬을 얻었습니다
-  아무래도 IT가 메인인 회사는 아니다보니 리테일 산업에 대한 이해와 IT를 여기서 어떻게 활용할 수 있을지에 대한 나만의 논리를 가지는 것이 중요했던 것 같습니다
-  아무 준비없이 서류, 인적성을 통과한다면 면접때 면접관이 하는 질문에 답하기가 어려울 것이라고 생각한다
-  그리고 면접이라는 것 자체가 같이 일하고 싶은 사람을 뽑는 자리이니만큼, 밝은 에너지를 보여드리며 자신감 있게 답변했던 것이 너무 당연하지만 주요한 합격 Tip이었던 것 같습니다
-  내가 합격한 이유는 자기소개서에 경험을 잘 어필했다는 것, 인성적으로 문제가 없게끔 서류부터 면접까지 잘 마무리 했다는 것인 것 같다

해당